In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Setup path per importare src
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src import config, models

# Verifica GPU
print(f"TensorFlow Version: {tf.__version__}")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-11-13 15:58:07.570481: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-13 15:58:08.074242: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-13 15:58:08.139424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-11-13 15:58:08.139481: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

TensorFlow Version: 2.10.1
Num GPUs Available:  0


2025-11-13 15:58:15.344708: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-11-13 15:58:15.344835: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (filippo-ideapad-slim-5): /proc/driver/nvidia/version does not exist


In [2]:
# Percorso del file generato nello step precedente
DATA_PATH = "../data/processed/train_class0_5k.npy"

# 1. Carica i dati
print(f"Caricamento dati da {DATA_PATH}...")
dataset_cadences = np.load(DATA_PATH)
print(f"Shape originale (Cadenze): {dataset_cadences.shape}")

# 2. Reshape: Da (N_samples, 6, 16, 512, 1) a (N_samples * 6, 16, 512, 1)
# Fondiamo la dimensione delle cadenze con quella dei campioni
# -1 dice a numpy di calcolare automaticamente la dimensione
X_all = dataset_cadences.reshape(-1, config.TCHANS, config.FCHANS_FINAL, 1)

print(f"Shape per il training (Frames singoli): {X_all.shape}")

# 3. Train/Test Split
# Usiamo il random_state per riproducibilità
X_train, X_val = train_test_split(X_all, test_size=0.2, random_state=42)

print(f"Training frames: {X_train.shape}")
print(f"Validation frames: {X_val.shape}")

Caricamento dati da ../data/processed/train_class0_5k.npy...
Shape originale (Cadenze): (5000, 6, 16, 512, 1)
Shape per il training (Frames singoli): (30000, 16, 512, 1)
Training frames: (24000, 16, 512, 1)
Validation frames: (6000, 16, 512, 1)


In [3]:
# Parametri
input_shape = (config.TCHANS, config.FCHANS_FINAL, 1)
latent_dim = config.LATENT_DIM # 8

print(f"Costruzione modello con input {input_shape} e latent dim {latent_dim}...")

# 1. Encoder & Decoder
encoder = models.build_encoder(input_shape, latent_dim)
decoder = models.build_decoder(latent_dim, target_shape=input_shape)

# 2. CVAE Completo
# beta=1.0 è un VAE standard.
# beta > 1 forza una rappresentazione latente più "districata" (disentangled)
cvae = models.CVAE(encoder, decoder, beta=1.0)

# Compile
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
cvae.compile(optimizer=optimizer)

# (Opzionale) Visualizza sommario encoder
encoder.summary()

Costruzione modello con input (16, 512, 1) e latent dim 64...


2025-11-13 16:09:58.578986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 512, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 8, 256, 32)   320         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 8, 256, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                            

In [ ]:
# Configurazione Training
EPOCHS = 20 
BATCH_SIZE = config.BATCH_SIZE # 32

print("Avvio training...")

history = cvae.fit(
    X_train, 
    validation_data=(X_val, X_val), # Nel VAE input e target sono uguali (ricostruzione)
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True
)

print("Training completato.")